### Cine21 영화배우 정보 크롤링
* site 주소: http://www.cine21.com/rank/person
* 개발자 도구에서 XHR인 것만 필터링해서 network -> content 페이지의 요청방식 확인
* Request URL: http://www.cine21.com/rank/person/content
* Request Method: POST
* Form Data (1개월치 데이터)
    - section: actor
    - period_start: 2020-07
    - gender: all
    - page: 1
* 1~2년치 데이터를 한꺼번에 크롤링해서 actors.json 파일로 저장하기


In [41]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import pymongo

### 정규표현식
- \w : 문자와 숫자
- * : 0~n (횟수), + : 1~n, ? : 0(zero) or 1

- .(period): 줄바꿈 문자(\n)를 제외한 모든 글자 1개를 의미
- .* : 문자가 0번 또는 그이상 반복되는 패턴
- Gredy(.*) vs Non-Greedy(.*?)
    - Greedy(.*) (욕심많은) : 모든 태그가 선택됨
    - Non-Greedy(.*?) (욕심없는) : 첫번째 매칭되는 태그만 선택
- [\n\t\r] : \n, \t, \r 중 하나를 찾는다
- re.compile(r'')  : 해당 정규표현식을 찾아주는 컴파일러를 생성해주는 함수
- compile() 함수 내에 정규표현식 앞에는 항상 r을 붙여줘야 한다

#### 1단계 : 상세페이지를 request해서 상세정보를 추출

In [108]:
actor_url = "http://www.cine21.com/rank/person/content"

formdata_dict = {
    'section': 'actor',
    'period_start': '2020-07',
    'gender': 'all',
    'page': 1
}

res = requests.post(actor_url, data=formdata_dict)
# print(res.ok)

soup = BeautifulSoup(res.text, 'html.parser')

actor_item_list = list()

for actor in soup.select('li.people_li div.name'):
#     print(actor.text.split('(')[0])
    print(re.sub('\(\w*\)','', actor.text))
    
for a_tag in soup.select('li.people_li div.name a'):
    actor_detail_url = urljoin(actor_url, a_tag['href'])
    
    res = requests.get(actor_detail_url)
    soup = BeautifulSoup(res.text, 'html.parser')

    act_item_dict = dict()
    for li_tag in soup.select('ul.default_info li'):
        # dict의 key값 추출 <li><span class="tit">직업</span>배우</li>
        actor_item_field = li_tag.select_one('span.tit').text

        # dict의 value값 추출 <li><span class="tit">직업</span>배우</li>
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) # <li>배우</li>
        actor_item_value = re.sub('<.*?>', '', actor_item_value) # 배우
        ''' 적용전
        http://www.gangdongwon.com

        '''
        regex = re.compile(r'[\n\r\t]') # 적용후 : http://www.gangdongwon.com
        actor_item_value = regex.sub('', actor_item_value)
        
        act_item_dict[actor_item_field] = actor_item_value
    actor_item_list.append(act_item_dict)
# actor_item_list

강동원
이정현
이레
권해효
구교환
김민재
유아인


In [12]:
test_data = '강동원(1편)'
re.sub('\(\w*\)','', test_data) # (문자가 있다면)

'강동원'

In [73]:
# 불필요한 공백들 제거
test_data = '''
http://www.gangdongwon.com

신장/체중
'''
regex = re.compile(r'[\n\r\t]')
regex.sub('', test_data)

'http://www.gangdongwon.com신장/체중'

#### 2단계 : 기본정보를 포함한 actor_dict 생성하고 list에 추가
#### 3단계 : 여러 페이지(1~3p)의 배우정보를 모두 추출 -> list에 추가

In [ ]:
actor_url = "http://www.cine21.com/rank/person/content"

formdata_dict = {
    'section': 'actor',
    'period_start': '2020-07',
    'gender': 'all',
    'page': 1
}

actor_item_list = list()

for page in range(1,4):
    formdata_dict['page'] = page
    print('---> {} 페이지 출력됨'.format(page))

    res = requests.post(actor_url, data=formdata_dict)
    # print(res.ok)

    soup = BeautifulSoup(res.text, 'html.parser')

    actors = soup.select('li.people_li div.name') # 배우
    hits = soup.select('ul.num_info > li > strong') # 흥행지수 # ul 바로 아래 있는 li tag 
    movies = soup.select('ul.mov_list') # 출연작
    rankings = soup.select('li.people_li span.grade') # 흥행순위

    # actor변수가 <div class=name>를 의미
    for idx, actor in enumerate(actors):
        print('===> {} 번째 배우 출력됨'.format(idx))
        actor_item_dict = dict()
    #     print(actor.text.split('(')[0])
        actor_name = re.sub('\(\w*\)','', actor.text)
        actor_item_dict['배우이름'] = actor_name
        actor_hit = int(hits[idx].text.replace(',', '')) # 흥행지수에서 ,(콤마) 제거후 정수로 변환
        actor_item_dict['흥행지수'] = actor_hit

        movie_titles = movies[idx].select('li span')
        movie_title_list = [] # 최대 5개의 출연작 title을 저장
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_item_dict['출연영화'] = movie_title_list

        actor_ranking = rankings[idx].text # 흥행순위
        actor_item_dict['랭킹'] = int(actor_ranking)

        # 배우의 상세정보(기본정보) 추출 
        # for a_tag in soup.select('li.people_li div.name a'):
        actor_detail_url = actor.select_one('a')['href']
        actor_detail_full_url = urljoin(actor_url, actor_detail_url)

        # 상세정보 페이지로 넘어가기 위해 다시 request
        res = requests.get(actor_detail_full_url)
        soup = BeautifulSoup(res.text, 'html.parser')

        for li_tag in soup.select('ul.default_info li'):
            # dict의 key값 추출 <li><span class="tit">직업</span>배우</li>
            actor_item_field = li_tag.select_one('span.tit').text

            # dict의 value값 추출 <li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) # <li>배우</li>
            actor_item_value = re.sub('<.*?>', '', actor_item_value) # 배우
            ''' 적용전
            http://www.gangdongwon.com

            '''
            regex = re.compile(r'[\n\r\t]') # 적용후 : http://www.gangdongwon.com
            actor_item_value = regex.sub('', actor_item_value)

            actor_item_dict[actor_item_field] = actor_item_value
        actor_item_list.append(actor_item_dict)

print(len(actor_item_list))
actor_item_list

#### 4단계 : 1년치 데이터 1905명의 흥행배우 정보를 모두 추출

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
# import pymongo
from itertools import count 
# from time import sleep

actor_url = "http://www.cine21.com/rank/person/content"

formdata_dict = {
    'section': 'actor',
    'period_start': '2019-07',
    'gender': 'all',
    'page': 1
}

actor_item_list = list()

# for page in range(1,4):
for page in count(1): # 무한루프
    formdata_dict['page'] = page
    print('---> {} 페이지 출력됨'.format(page))

    res = requests.post(actor_url, data=formdata_dict)
    # print(res.ok)

    soup = BeautifulSoup(res.text, 'html.parser')

    actors = soup.select('li.people_li div.name') # 배우
    if len(actors) == 0:
        break
    hits = soup.select('ul.num_info > li > strong') # 흥행지수 # ul 바로 아래 있는 li tag 
    movies = soup.select('ul.mov_list') # 출연작
    rankings = soup.select('li.people_li span.grade') # 흥행순위

    # actor변수가 <div class=name>를 의미
    for idx, actor in enumerate(actors):
        actor_item_dict = dict()
    #     print(actor.text.split('(')[0])
        actor_name = re.sub('\(\w*\)','', actor.text)
        actor_item_dict['배우이름'] = actor_name
        print('===> {}({}) 배우 출력됨'.format(actor_name, len(actor_name)))
        actor_hit = int(hits[idx].text.replace(',', '')) # 흥행지수에서 ,(콤마) 제거후 정수로 변환
        actor_item_dict['흥행지수'] = actor_hit

        movie_titles = movies[idx].select('li span')
        movie_title_list = [] # 최대 5개의 출연작 title을 저장
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_item_dict['출연영화'] = movie_title_list

        actor_ranking = rankings[idx].text # 흥행순위
        actor_item_dict['랭킹'] = int(actor_ranking)

        # 배우의 상세정보(기본정보) 추출 
        # for a_tag in soup.select('li.people_li div.name a'):
        actor_detail_url = actor.select_one('a')['href']
        actor_detail_full_url = urljoin(actor_url, actor_detail_url)

        # 상세정보 페이지로 넘어가기 위해 다시 request
        res = requests.get(actor_detail_full_url)
        soup = BeautifulSoup(res.text, 'html.parser')

        for li_tag in soup.select('ul.default_info li'):
            # dict의 key값 추출 <li><span class="tit">직업</span>배우</li>
            actor_item_field = li_tag.select_one('span.tit').text

            # dict의 value값 추출 <li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) # <li>배우</li>
            actor_item_value = re.sub('<.*?>', '', actor_item_value) # 배우
            ''' 적용전
            http://www.gangdongwon.com

            '''
            regex = re.compile(r'[\n\r\t]') # 적용후 : http://www.gangdongwon.com
            actor_item_value = regex.sub('', actor_item_value)

            actor_item_dict[actor_item_field] = actor_item_value
        actor_item_list.append(actor_item_dict)
#         sleep(0.3)

---> 1 페이지 출력됨
===> 이병헌(3) 배우 출력됨
===> 하정우(3) 배우 출력됨
===> 마동석(3) 배우 출력됨
===> 전혜진(3) 배우 출력됨
===> 이성민(3) 배우 출력됨
===> 김아중(3) 배우 출력됨
===> 박정민(3) 배우 출력됨
---> 2 페이지 출력됨
===> 강기영(3) 배우 출력됨
===> 조진웅(3) 배우 출력됨
===> 조정석(3) 배우 출력됨
===> 배수지(3) 배우 출력됨
===> 유아인(3) 배우 출력됨
===> 임윤아(3) 배우 출력됨
===> 고두심(3) 배우 출력됨
---> 3 페이지 출력됨
===> 장기용(3) 배우 출력됨
===> 권상우(3) 배우 출력됨
===> 곽도원(3) 배우 출력됨
===> 유해진(3) 배우 출력됨
===> 최유화(3) 배우 출력됨
===> 박인환(3) 배우 출력됨
===> 정해인(3) 배우 출력됨
---> 4 페이지 출력됨
===> 정유미(3) 배우 출력됨
===> 김지영(3) 배우 출력됨
===> 류준열(3) 배우 출력됨
===> 이봉련(3) 배우 출력됨
===> 조우진(3) 배우 출력됨
===> 공유(2) 배우 출력됨
===> 강동원(3) 배우 출력됨
---> 5 페이지 출력됨
===> 김종구(3) 배우 출력됨
===> 박지환(3) 배우 출력됨
===> 이정현(3) 배우 출력됨
===> 김상중(3) 배우 출력됨
===> 이희준(3) 배우 출력됨
===> 김미경(3) 배우 출력됨
===> 허성태(3) 배우 출력됨
---> 6 페이지 출력됨
===> 김병순(3) 배우 출력됨
===> 박신혜(3) 배우 출력됨
===> 염정아(3) 배우 출력됨
===> 배성우(3) 배우 출력됨
===> 이경영(3) 배우 출력됨
===> 허준호(3) 배우 출력됨
===> 권해효(3) 배우 출력됨
---> 7 페이지 출력됨
===> 황효은(3) 배우 출력됨
===> 이레(2) 배우 출력됨
===> 이하늬(3) 배우 출력됨
===> 김래원(3) 배우 출력됨
===> 공민정(3) 배우 출력됨
===>

===> 이주원(3) 배우 출력됨
===> 김호정(3) 배우 출력됨
===> 최정윤(3) 배우 출력됨
===> 김민상(3) 배우 출력됨
===> 박효주(3) 배우 출력됨
===> 신은수(3) 배우 출력됨
---> 57 페이지 출력됨
===> 김동완(3) 배우 출력됨
===> 안장혁(3) 배우 출력됨
===> 이관희(3) 배우 출력됨
===> 장다운(3) 배우 출력됨
===> 유이화(3) 배우 출력됨
===> 송창의(3) 배우 출력됨
===> 주예림(3) 배우 출력됨
---> 58 페이지 출력됨
===> 김민준(3) 배우 출력됨
===> 이금희(3) 배우 출력됨
===> 최무성(3) 배우 출력됨
===> 이준혁(3) 배우 출력됨
===> 강말금(3) 배우 출력됨
===> 박소이(3) 배우 출력됨
===> 변영희(3) 배우 출력됨
---> 59 페이지 출력됨
===> 안지호(3) 배우 출력됨
===> 주진모(3) 배우 출력됨
===> 오현경(3) 배우 출력됨
===> 성훈(2) 배우 출력됨
===> 천정명(3) 배우 출력됨
===> 이혜리(3) 배우 출력됨
===> 김사라(3) 배우 출력됨
---> 60 페이지 출력됨
===> 손숙(2) 배우 출력됨
===> 김서영(3) 배우 출력됨
===> 문소리(3) 배우 출력됨
===> 정형석(3) 배우 출력됨
===> 염혜란(3) 배우 출력됨
===> 김영민(3) 배우 출력됨
===> 김민주(3) 배우 출력됨
---> 61 페이지 출력됨
===> 이성우(3) 배우 출력됨
===> 정재현(3) 배우 출력됨
===> 송하림(3) 배우 출력됨
===> 전수지(3) 배우 출력됨
===> 조민수(3) 배우 출력됨
===> 오나미(3) 배우 출력됨
===> 이봉근(3) 배우 출력됨
---> 62 페이지 출력됨
===> 조진숙(3) 배우 출력됨
===> 진이한(3) 배우 출력됨
===> 허가윤(3) 배우 출력됨
===> 최덕희(3) 배우 출력됨
===> 임형준(3) 배우 출력됨
===> 이유영(3) 배우 출력됨
===> 오은주(3) 배

===> 홍승이(3) 배우 출력됨
===> 김규리(3) 배우 출력됨
===> 현빈(2) 배우 출력됨
===> 신린아(3) 배우 출력됨
===> 이설구(3) 배우 출력됨
===> 김현(2) 배우 출력됨
===> 김나니(3) 배우 출력됨
---> 111 페이지 출력됨
===> 김순규(3) 배우 출력됨
===> 신철진(3) 배우 출력됨
===> 임권택(3) 배우 출력됨
===> 송재호(3) 배우 출력됨
===> 이창성(3) 배우 출력됨
===> 박자희(3) 배우 출력됨
===> 정다원(3) 배우 출력됨
---> 112 페이지 출력됨
===> 정다원(3) 배우 출력됨
===> 오창석(3) 배우 출력됨
===> 형영선(3) 배우 출력됨
===> 권소현(3) 배우 출력됨
===> 권태원(3) 배우 출력됨
===> 이용한(3) 배우 출력됨
===> 김라희(3) 배우 출력됨
---> 113 페이지 출력됨
===> 강신성일(4) 배우 출력됨
===> 이효제(3) 배우 출력됨
===> 박지아(3) 배우 출력됨
===> 서정연(3) 배우 출력됨
===> 이나라(3) 배우 출력됨
===> 엄앵란(3) 배우 출력됨
===> 양윤희(3) 배우 출력됨
---> 114 페이지 출력됨
===> 이선(2) 배우 출력됨
===> 최홍일(3) 배우 출력됨
===> 김대건(3) 배우 출력됨
===> 윤정욱(3) 배우 출력됨
===> 유상재(3) 배우 출력됨
===> 조영진(3) 배우 출력됨
===> 김태희(3) 배우 출력됨
---> 115 페이지 출력됨
===> 김현목(3) 배우 출력됨
===> 강두(2) 배우 출력됨
===> 백윤식(3) 배우 출력됨
===> 이진이(3) 배우 출력됨
===> 이대엽(3) 배우 출력됨
===> 남정임(3) 배우 출력됨
===> 류의도(3) 배우 출력됨
---> 116 페이지 출력됨
===> 윤진서(3) 배우 출력됨
===> 탁우석(3) 배우 출력됨
===> 차종수(3) 배우 출력됨
===> 이대근(3) 배우 출력됨
===> 류경수(3) 배우 출력됨
===> 김막동

---> 165 페이지 출력됨
===> 남명렬(3) 배우 출력됨
===> 이학주(3) 배우 출력됨
===> 김태훈(3) 배우 출력됨
===> 노지유(3) 배우 출력됨
===> 하휘동(3) 배우 출력됨
===> 이지연(3) 배우 출력됨
===> 김여진(3) 배우 출력됨
---> 166 페이지 출력됨
===> 김여진(3) 배우 출력됨
===> 정애란(3) 배우 출력됨
===> 문가영(3) 배우 출력됨
===> 장예나(3) 배우 출력됨
===> 김인문(3) 배우 출력됨
===> 이재웅(3) 배우 출력됨
===> 전무송(3) 배우 출력됨
---> 167 페이지 출력됨
===> 양훈(2) 배우 출력됨
===> 전무송(3) 배우 출력됨
===> 서진원(3) 배우 출력됨
===> 김선경(3) 배우 출력됨
===> 백도빈(3) 배우 출력됨
===> 최진욱(3) 배우 출력됨
===> 




씨네21









서비스 이용에 불편을 드려 죄송합니다.
일시적으로 서비스 장애가 발생하였습니다.
빠른 시간안에 복구하도록 하겠습니다.
			





">




씨네21









서비스 이용에 불편을 드려 죄송합니다.
일시적으로 서비스 장애가 발생하였습니다.
빠른 시간안에 복구하도록 하겠습니다.
			





(190) 배우 출력됨
---> 168 페이지 출력됨
===> 김복자(3) 배우 출력됨
===> 김동욱(3) 배우 출력됨
===> 김결(2) 배우 출력됨
===> 임현식(3) 배우 출력됨
===> 이제연(3) 배우 출력됨
===> 백일섭(3) 배우 출력됨
===> 정우식(3) 배우 출력됨
---> 169 페이지 출력됨
===> 강계식(3) 배우 출력됨
===> 홍기(2) 배우 출력됨
===> 원세훈(3) 배우 출력됨
===> 노율(2) 배우 출력됨
===> 최리(2) 배우 출력됨
===> 엄지원(3) 배우 출력됨
===> 정하담(3) 배우 출력됨
---> 170 페이지 출력됨
===> 이현행(3) 배우 출력됨
===> 곽인준(3) 배우 출력됨
===> 민채은(3)

===> 구웅서(3) 배우 출력됨
===> 김영진(3) 배우 출력됨
---> 219 페이지 출력됨
===> 이완(2) 배우 출력됨
===> 이요원(3) 배우 출력됨
===> 윤지혜(3) 배우 출력됨
===> 이달형(3) 배우 출력됨
===> 양홍석(3) 배우 출력됨
===> 하박(2) 배우 출력됨
===> 이소영(3) 배우 출력됨
---> 220 페이지 출력됨
===> 허기호(3) 배우 출력됨
===> 홍윤정(3) 배우 출력됨
===> 박세진(3) 배우 출력됨
===> 나영희(3) 배우 출력됨
===> 최휘찬(3) 배우 출력됨
===> 박은숙(3) 배우 출력됨
===> 이소영(3) 배우 출력됨
---> 221 페이지 출력됨
===> 문석범(3) 배우 출력됨
===> 정기섭(3) 배우 출력됨
===> 김도엽(3) 배우 출력됨
===> 진세연(3) 배우 출력됨
===> 윤가현(3) 배우 출력됨
===> 황윤(2) 배우 출력됨
===> 이보영(3) 배우 출력됨
---> 222 페이지 출력됨
===> 천보근(3) 배우 출력됨
===> 허기호(3) 배우 출력됨
===> 홍윤정(3) 배우 출력됨
===> 최휘찬(3) 배우 출력됨
===> 박세진(3) 배우 출력됨
===> 나영희(3) 배우 출력됨
===> 박은숙(3) 배우 출력됨
---> 223 페이지 출력됨
===> 서강준(3) 배우 출력됨
===> 이일웅(3) 배우 출력됨
===> 노민우(3) 배우 출력됨
===> 박형수(3) 배우 출력됨
===> 박정자(3) 배우 출력됨
===> 이승신(3) 배우 출력됨
===> 배제기(3) 배우 출력됨
---> 224 페이지 출력됨
===> 박성우(3) 배우 출력됨
===> 백선혁(3) 배우 출력됨
===> 김서원(3) 배우 출력됨
===> 유계선(3) 배우 출력됨
===> 한지은(3) 배우 출력됨
===> 서강준(3) 배우 출력됨
===> 이일웅(3) 배우 출력됨
---> 225 페이지 출력됨
===> 배제기(3) 배우 출력됨
===> 이기영(3) 배우 출력됨
===> 홍상표(3

In [2]:
print(len(actor_item_list))
actor_final_list = [actor for actor in actor_item_list if len(actor['배우이름']) < 10]
print(len(actor_final_list))

1905
1903


### json file로 저장

In [6]:
import json

with open('data/actors.json', 'w', encoding='utf-8') as file:
    json.dump(actor_final_list, file, ensure_ascii=False)

print('actors.json 저장 완료')

actors.json 저장 완료
